## Overview
Saves blocks to local `blocks.json` files:
1. Loads blocks using getBlocks gRPC requests from pruning point to selected tip.
2. Gets virtual selected parent chain via gRPC. Updates chainblock and tx acceptance status.
3. Saves to json file

In [1]:
from collections import defaultdict, OrderedDict
from datetime import datetime
import json

from IPython.display import clear_output
from kaspad.KaspadClient import KaspadClient

## Helpers

In [2]:
async def get_blocks(rpc_client, low_hash):
    r = await rpc_client.request(
        "getBlocksRequest",
        params={
            "lowHash": low_hash,
            "includeTransactions": True,
            "includeBlocks": True
        },
    )
    return r['getBlocksResponse']

In [3]:
async def get_vspc(rpc_client, low_hash):
    r = await rpc_client.request(
        'getVirtualSelectedParentChainFromBlockRequest',
        params={
            'startHash': low_hash,
            'includeAcceptedTransactionIds': True,
        },
        timeout=60 * 60 * 1 # seconds * minutes * hours
    )
    return r['getVirtualSelectedParentChainFromBlockResponse']

In [4]:
async def get_dag_info(rpc_client):
    r = await rpc_client.request('getBlockDagInfoRequest')
    return r['getBlockDagInfoResponse']

In [5]:
async def get_selected_tip(rpc_client):
    r = await rpc_client.request('getSelectedTipHashRequest')
    return r['getSelectedTipHashResponse']

## Main

In [6]:
rpc_client = KaspadClient('localhost', 16120)

In [7]:
# Get pruning point hash
dag_info = await get_dag_info(rpc_client)
low_hash = dag_info['pruningPointHash']

In [8]:
# Load blocks from pruning point hash to tip
block_cache = OrderedDict()
tx_to_blocks_index = defaultdict(list)

while True:
    blocks = await get_blocks(rpc_client, low_hash)

    for idx, block in enumerate(blocks.get('blocks', [])):
        hash = block['verboseData']['hash']

        # Keep 1 level of parents for memory/storage purposes
        block['header']['parents'] = block['header']['parents'][0]['parentHashes']

        # Add block to cache
        block_cache[hash] = block

        # Store tx to blocks mapping
        for tx in block['transactions']:
            tx_id = tx['verboseData']['transactionId']

            tx_to_blocks_index[tx_id].append(hash)

        # Break once we get to DAG tip
        selected_tip = await get_selected_tip(rpc_client)
        if selected_tip['selectedTipHash'] == hash:
            break
        
        low_hash = hash

    last_cache_block = next(reversed(block_cache))

    # Break once we get to DAG tip
    selected_tip = await get_selected_tip(rpc_client)
    if selected_tip['selectedTipHash'] == last_cache_block:
        break

    # keep an eye on progress
    clear_output(wait=True)
    print(last_cache_block, datetime.fromtimestamp( int(block_cache[last_cache_block]['header']['timestamp']) / 1000 ))

04ce38393e6c858589d998f320c86a9a81e04c5398a667af76a87a47afd31d84 2023-12-14 18:50:21.686000


In [9]:
# Apply virtual selected parent chain to block_cache

vspc_low_hash = next(iter(block_cache))
vspc = await get_vspc(rpc_client, vspc_low_hash)

# Block hash to stop VSPC iteration at to ensure accuracy
vspc_stop_hash = None
for k, v in reversed(block_cache.items()):
    if v['verboseData']['isChainBlock']:
        vspc_stop_hash = k
        break

# set isChainBlock to False for removed blocks
for hash in vspc.get("removedChainBlockHashes", []):
    block_cache[hash]['verboseData']['isChainBlock'] = False

# set isChainBlock to True for added blocks
for hash in vspc.get("addedChainBlockHashes", []):
    if hash == vspc_stop_hash:
        break
    
    block_cache[hash]['verboseData']['isChainBlock'] = True

# set accepted to True to for accepted transactions
tx_in_blocks_none = 0
for d in vspc.get('acceptedTransactionIds', []):
    if d['acceptingBlockHash'] == vspc_stop_hash:
        break
        
    for accepted_tx_id in d['acceptedTransactionIds']:
        tx_in_blocks = tx_to_blocks_index.get(accepted_tx_id)

        if tx_in_blocks is None:
            # TODO ???
            tx_in_blocks_none += 1
            continue

        for block_hash in tx_in_blocks:
            for i, tx in enumerate(block_cache[block_hash]['transactions']):
                if tx['verboseData']['transactionId'] == accepted_tx_id:
                    block_cache[block_hash]['transactions'][i]['accepted'] = True
                    block_cache[block_hash]['transactions'][i]['acceptingBlockHash'] = d['acceptingBlockHash']

In [12]:
# Dump to JSON
with open('block.json', 'w') as f:
    json.dump({'blocks': block_cache}, f, indent=4)